# Air Quality Forecasting - Complete Workflow

This notebook demonstrates the complete workflow for short-term air quality forecasting using satellite and reanalysis data.

## Overview
- **Objective**: Forecast ground-level O3 and NO2 concentrations 24-48 hours ahead
- **Data**: Meteorological forecasts, satellite observations, and ground truth measurements
- **Models**: Random Forest, XGBoost, LSTM, CNN-LSTM
- **Target**: Hourly predictions with confidence intervals


In [ ]:
# Import necessary libraries
import sys
import os
from pathlib import Path

# Add src directory to path
project_root = Path.cwd().parent
sys.path.append(str(project_root / 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Project root: {project_root}")

In [ ]:
# Import project modules
from data_preprocessing.data_loader import DataLoader
from feature_engineering.feature_engineer import FeatureEngineer
from models.model_trainer import ModelTrainer
from evaluation.model_evaluator import ModelEvaluator
from forecasting.forecaster import AirQualityForecaster
from utils.helpers import ConfigManager, LoggingUtils, DataUtils

print("Project modules imported successfully!")

## 1. Configuration and Setup

In [ ]:
# Load configuration
config_path = project_root / 'configs' / 'config.yaml'
config = ConfigManager.load_config(str(config_path))

print("Configuration loaded:")
print(f"- Project: {config.get('project', {}).get('name', 'N/A')}")
print(f"- Version: {config.get('project', {}).get('version', 'N/A')}")
print(f"- Data path: {config.get('data', {}).get('raw_data_path', 'N/A')}")
print(f"- Models: {list(config.get('models', {}).keys())}")

In [ ]:
# Setup logging
logger = LoggingUtils.setup_logger(
    name="air_quality_forecast",
    level="INFO",
    log_file=str(project_root / "logs" / "training.log")
)

logger.info("Starting air quality forecasting workflow")

## 2. Data Loading and Exploration

In [ ]:
# Initialize data loader
data_loader = DataLoader(config_path=str(config_path))

# Load site coordinates
coords_df = data_loader.load_site_coordinates()
print("Site Coordinates:")
display(coords_df)

In [ ]:
# Load training data for all sites
train_data_dict = data_loader.load_training_data()
print(f"Loaded training data for {len(train_data_dict)} sites")

# Display data summary for each site
for site_id, data in train_data_dict.items():
    summary = data_loader.get_data_summary(data)
    print(f"\nSite {site_id}:")
    print(f"  Shape: {summary['shape']}")
    print(f"  Date range: {summary['date_range']['start']} to {summary['date_range']['end']}")
    print(f"  Duration: {summary['date_range']['duration_days']} days")

In [ ]:
# Examine data structure for Site 1
site_1_data = train_data_dict[1]
print("Sample data structure (Site 1):")
print(f"Columns: {list(site_1_data.columns)}")
print(f"\nFirst few rows:")
display(site_1_data.head())

print(f"\nData types:")
display(site_1_data.dtypes)

In [ ]:
# Visualize data availability and missing values
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot target variables time series for Site 1
axes[0, 0].plot(site_1_data['datetime'], site_1_data['O3_target'], alpha=0.7, label='O3')
axes[0, 0].set_title('Ground-level O3 (Site 1)')
axes[0, 0].set_ylabel('O3 (μg/m³)')
axes[0, 0].legend()

axes[0, 1].plot(site_1_data['datetime'], site_1_data['NO2_target'], alpha=0.7, label='NO2', color='orange')
axes[0, 1].set_title('Ground-level NO2 (Site 1)')
axes[0, 1].set_ylabel('NO2 (μg/m³)')
axes[0, 1].legend()

# Missing data heatmap
missing_data = site_1_data.isnull().sum()
missing_pct = (missing_data / len(site_1_data) * 100).sort_values(ascending=False)
missing_pct[missing_pct > 0].plot(kind='bar', ax=axes[1, 0])
axes[1, 0].set_title('Missing Data Percentage (Site 1)')
axes[1, 0].set_ylabel('% Missing')
axes[1, 0].tick_params(axis='x', rotation=45)

# Correlation matrix of key variables
key_vars = ['O3_target', 'NO2_target', 'T_forecast', 'q_forecast', 'O3_forecast', 'NO2_forecast']
corr_matrix = site_1_data[key_vars].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1, 1])
axes[1, 1].set_title('Correlation Matrix (Site 1)')

plt.tight_layout()
plt.show()

## 3. Data Preprocessing and Cleaning

In [ ]:
# Clean and preprocess data for all sites
cleaned_data_dict = {}

for site_id, data in train_data_dict.items():
    print(f"Cleaning data for Site {site_id}...")
    
    # Clean the data
    cleaned_data = data_loader.clean_data(data)
    
    # Prepare features and targets
    features_df, targets_df = data_loader.prepare_features_targets(cleaned_data)
    
    # Combine for storage
    combined_df = pd.concat([features_df, targets_df], axis=1)
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]  # Remove duplicate columns
    
    cleaned_data_dict[site_id] = combined_df
    
    print(f"  Original shape: {data.shape}")
    print(f"  Cleaned shape: {cleaned_data.shape}")
    print(f"  Final shape: {combined_df.shape}")

print("\nData cleaning completed for all sites.")

## 4. Feature Engineering

In [ ]:
# Initialize feature engineer
feature_engineer = FeatureEngineer(config_path=str(config_path))

# Apply feature engineering to all sites
engineered_data_dict = {}

for site_id, data in cleaned_data_dict.items():
    print(f"Engineering features for Site {site_id}...")
    
    # Apply comprehensive feature engineering
    engineered_data = feature_engineer.create_all_features(data)
    engineered_data_dict[site_id] = engineered_data
    
    print(f"  Original features: {data.shape[1]}")
    print(f"  Engineered features: {engineered_data.shape[1]}")
    print(f"  New features added: {engineered_data.shape[1] - data.shape[1]}")

print("\nFeature engineering completed for all sites.")

In [ ]:
# Show feature groups
feature_groups = feature_engineer.get_feature_importance_groups()
print("Feature Groups:")
for group_name, features in feature_groups.items():
    if isinstance(features, list):
        print(f"  {group_name}: {len(features)} features")
    else:
        # Pattern-based features
        pattern_features = [col for col in engineered_data_dict[1].columns if features in col]
        print(f"  {group_name}: {len(pattern_features)} features (pattern: '{features}')")

In [ ]:
# Combine all sites for model training
all_sites_data = data_loader.combine_all_sites(engineered_data_dict)
print(f"Combined dataset shape: {all_sites_data.shape}")
print(f"Date range: {all_sites_data['datetime'].min()} to {all_sites_data['datetime'].max()}")
print(f"Sites included: {sorted(all_sites_data['site_id'].unique())}")

## 5. Model Training

In [ ]:
# Initialize model trainer
model_trainer = ModelTrainer(config_path=str(config_path))

# Prepare data for O3 prediction
print("Preparing data for O3 model training...")
X_o3, y_o3, feature_names = model_trainer.prepare_training_data(
    all_sites_data, 
    all_sites_data[['O3_target', 'datetime', 'site_id']], 
    'O3_target'
)

print(f"O3 training data prepared:")
print(f"  Features shape: {X_o3.shape}")
print(f"  Targets shape: {y_o3.shape}")
print(f"  Number of features: {len(feature_names)}")

In [ ]:
# Split data
data_splits_o3 = model_trainer.split_data(X_o3, y_o3)

print("Data splits for O3:")
for split_name, data in data_splits_o3.items():
    print(f"  {split_name}: {data.shape}")

In [ ]:
# Train all models for O3
print("Training all models for O3 prediction...")
o3_models = model_trainer.train_all_models(
    data_splits_o3['X_train'], data_splits_o3['y_train'],
    data_splits_o3['X_val'], data_splits_o3['y_val']
)

print(f"\nTrained {len(o3_models)} models for O3:")
for model_name in o3_models.keys():
    print(f"  - {model_name}")

In [ ]:
# Train models for NO2
print("Preparing data for NO2 model training...")
X_no2, y_no2, _ = model_trainer.prepare_training_data(
    all_sites_data, 
    all_sites_data[['NO2_target', 'datetime', 'site_id']], 
    'NO2_target'
)

data_splits_no2 = model_trainer.split_data(X_no2, y_no2)

print("Training all models for NO2 prediction...")
no2_models = model_trainer.train_all_models(
    data_splits_no2['X_train'], data_splits_no2['y_train'],
    data_splits_no2['X_val'], data_splits_no2['y_val']
)

print(f"\nTrained {len(no2_models)} models for NO2:")
for model_name in no2_models.keys():
    print(f"  - {model_name}")

## 6. Model Evaluation

In [ ]:
# Initialize model evaluator
evaluator = ModelEvaluator(config_path=str(config_path))

# Evaluate O3 models
print("Evaluating O3 models...")
o3_comparison = evaluator.compare_models(
    o3_models, 
    data_splits_o3['X_test'], 
    data_splits_o3['y_test'], 
    target_name="O3"
)

print("\nO3 Model Comparison Results:")
display(o3_comparison[['model_name', 'r2', 'rmse', 'mae', 'correlation']].round(4))

In [ ]:
# Evaluate NO2 models
print("Evaluating NO2 models...")
no2_comparison = evaluator.compare_models(
    no2_models, 
    data_splits_no2['X_test'], 
    data_splits_no2['y_test'], 
    target_name="NO2"
)

print("\nNO2 Model Comparison Results:")
display(no2_comparison[['model_name', 'r2', 'rmse', 'mae', 'correlation']].round(4))

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# O3 model comparison
o3_comparison.plot(x='model_name', y='r2', kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('O3 Model Performance (R²)')
axes[0].set_ylabel('R² Score')
axes[0].tick_params(axis='x', rotation=45)

# NO2 model comparison
no2_comparison.plot(x='model_name', y='r2', kind='bar', ax=axes[1], color='lightcoral')
axes[1].set_title('NO2 Model Performance (R²)')
axes[1].set_ylabel('R² Score')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Create detailed plots for best performing models
best_o3_model_name = o3_comparison.iloc[0]['model_name']
best_no2_model_name = no2_comparison.iloc[0]['model_name']

print(f"Best O3 model: {best_o3_model_name}")
print(f"Best NO2 model: {best_no2_model_name}")

# Get predictions from best models
best_o3_model = o3_models[best_o3_model_name]
best_no2_model = no2_models[best_no2_model_name]

# Generate predictions
is_o3_sequence = 'lstm' in best_o3_model_name.lower()
is_no2_sequence = 'lstm' in best_no2_model_name.lower()

if is_o3_sequence:
    X_test_seq, y_test_seq = evaluator._prepare_sequence_data(
        data_splits_o3['X_test'], data_splits_o3['y_test'], 24
    )
    o3_predictions = best_o3_model.predict(X_test_seq).flatten()
    y_true_o3 = y_test_seq
else:
    o3_predictions = best_o3_model.predict(data_splits_o3['X_test'])
    y_true_o3 = data_splits_o3['y_test']

if is_no2_sequence:
    X_test_seq, y_test_seq = evaluator._prepare_sequence_data(
        data_splits_no2['X_test'], data_splits_no2['y_test'], 24
    )
    no2_predictions = best_no2_model.predict(X_test_seq).flatten()
    y_true_no2 = y_test_seq
else:
    no2_predictions = best_no2_model.predict(data_splits_no2['X_test'])
    y_true_no2 = data_splits_no2['y_test']

In [ ]:
# Plot predictions vs actual for best models
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# O3 predictions vs actual
axes[0].scatter(y_true_o3, o3_predictions, alpha=0.6, s=30)
min_val = min(np.min(y_true_o3), np.min(o3_predictions))
max_val = max(np.max(y_true_o3), np.max(o3_predictions))
axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual O3')
axes[0].set_ylabel('Predicted O3')
axes[0].set_title(f'O3 Predictions vs Actual ({best_o3_model_name})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# NO2 predictions vs actual
axes[1].scatter(y_true_no2, no2_predictions, alpha=0.6, s=30, color='orange')
min_val = min(np.min(y_true_no2), np.min(no2_predictions))
max_val = max(np.max(y_true_no2), np.max(no2_predictions))
axes[1].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual NO2')
axes[1].set_ylabel('Predicted NO2')
axes[1].set_title(f'NO2 Predictions vs Actual ({best_no2_model_name})')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Feature importance for tree-based models
if hasattr(best_o3_model, 'feature_importances_'):
    print(f"Feature importance for {best_o3_model_name} (O3):")
    evaluator.plot_feature_importance(
        best_o3_model, feature_names, 
        model_name=f"{best_o3_model_name} (O3)", 
        top_k=15
    )

if hasattr(best_no2_model, 'feature_importances_'):
    print(f"Feature importance for {best_no2_model_name} (NO2):")
    evaluator.plot_feature_importance(
        best_no2_model, feature_names, 
        model_name=f"{best_no2_model_name} (NO2)", 
        top_k=15
    )

## 7. Generate Forecasts

In [ ]:
# Load test data for forecasting
test_data_dict = data_loader.load_test_data()
print(f"Loaded test data for {len(test_data_dict)} sites")

# Apply same preprocessing and feature engineering to test data
processed_test_data = {}

for site_id, data in test_data_dict.items():
    print(f"Processing test data for Site {site_id}...")
    
    # Clean and engineer features
    cleaned_data = data_loader.clean_data(data)
    features_df, _ = data_loader.prepare_features_targets(cleaned_data)
    engineered_data = feature_engineer.create_all_features(features_df)
    
    processed_test_data[site_id] = engineered_data
    print(f"  Processed shape: {engineered_data.shape}")

In [ ]:
# Initialize forecaster
forecaster = AirQualityForecaster(config_path=str(config_path))

# Load best models for forecasting
best_models = {
    'O3_' + best_o3_model_name: best_o3_model,
    'NO2_' + best_no2_model_name: best_no2_model
}

# Load scalers if available
scalers = model_trainer.scalers if hasattr(model_trainer, 'scalers') else {}

forecaster.load_models(best_models, scalers, feature_names)
print(f"Forecaster status: {forecaster.get_forecast_status()}")

In [ ]:
# Generate forecasts for Site 1 as example
site_id = 1
site_test_data = processed_test_data[site_id]

print(f"Generating forecasts for Site {site_id}...")

# Generate 24-hour forecasts
site_forecasts = forecaster.forecast_for_site(
    site_test_data, 
    site_id,
    target_columns=['O3_target', 'NO2_target'],
    forecast_hours=24
)

print(f"Generated forecasts for {len(site_forecasts)} targets")
for target, forecast_df in site_forecasts.items():
    print(f"  {target}: {len(forecast_df)} hourly predictions")

In [ ]:
# Display forecast results
print("Sample O3 forecasts (first 10 hours):")
if 'O3_target' in site_forecasts:
    display(site_forecasts['O3_target'].head(10))

print("\nSample NO2 forecasts (first 10 hours):")
if 'NO2_target' in site_forecasts:
    display(site_forecasts['NO2_target'].head(10))

In [ ]:
# Visualize forecasts
if site_forecasts:
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    # O3 forecast
    if 'O3_target' in site_forecasts:
        o3_forecast = site_forecasts['O3_target']
        axes[0].plot(o3_forecast['datetime'], o3_forecast['O3_target_prediction'], 
                    label='O3 Prediction', linewidth=2)
        axes[0].fill_between(o3_forecast['datetime'], 
                           o3_forecast['O3_target_lower_ci'], 
                           o3_forecast['O3_target_upper_ci'], 
                           alpha=0.3, label='90% Confidence Interval')
        axes[0].set_title(f'O3 Forecast for Site {site_id} (24 hours)')
        axes[0].set_ylabel('O3 (μg/m³)')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
    
    # NO2 forecast
    if 'NO2_target' in site_forecasts:
        no2_forecast = site_forecasts['NO2_target']
        axes[1].plot(no2_forecast['datetime'], no2_forecast['NO2_target_prediction'], 
                    label='NO2 Prediction', linewidth=2, color='orange')
        axes[1].fill_between(no2_forecast['datetime'], 
                           no2_forecast['NO2_target_lower_ci'], 
                           no2_forecast['NO2_target_upper_ci'], 
                           alpha=0.3, label='90% Confidence Interval', color='orange')
        axes[1].set_title(f'NO2 Forecast for Site {site_id} (24 hours)')
        axes[1].set_ylabel('NO2 (μg/m³)')
        axes[1].set_xlabel('DateTime')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 8. Save Results and Models

In [ ]:
# Create results directories
results_dir = project_root / 'results'
models_dir = project_root / 'models'
results_dir.mkdir(exist_ok=True)
models_dir.mkdir(exist_ok=True)

# Save model comparison results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

o3_comparison.to_csv(results_dir / f'o3_model_comparison_{timestamp}.csv', index=False)
no2_comparison.to_csv(results_dir / f'no2_model_comparison_{timestamp}.csv', index=False)

print(f"Model comparison results saved to {results_dir}")

In [ ]:
# Save best models
model_trainer.save_model(best_o3_model_name, str(models_dir / f'best_o3_model_{timestamp}'))
model_trainer.save_model(best_no2_model_name, str(models_dir / f'best_no2_model_{timestamp}'))

print(f"Best models saved to {models_dir}")

In [ ]:
# Save forecasts
if site_forecasts:
    forecaster.save_forecasts(
        site_forecasts,
        output_dir=str(results_dir / 'forecasts'),
        timestamp=timestamp
    )
    print(f"Forecasts saved to {results_dir / 'forecasts'}")

In [ ]:
# Generate comprehensive evaluation report
o3_report = evaluator.create_comprehensive_report(
    o3_comparison, 
    target_name="Ground-level O3",
    save_path=str(results_dir / f'o3_evaluation_report_{timestamp}.md')
)

no2_report = evaluator.create_comprehensive_report(
    no2_comparison, 
    target_name="Ground-level NO2",
    save_path=str(results_dir / f'no2_evaluation_report_{timestamp}.md')
)

print("Comprehensive evaluation reports generated.")

## 9. Summary and Conclusions

In [ ]:
# Print final summary
print("=" * 60)
print("AIR QUALITY FORECASTING - WORKFLOW SUMMARY")
print("=" * 60)

print(f"\n1. DATA PROCESSING:")
print(f"   - Sites processed: {len(train_data_dict)}")
print(f"   - Training samples: {len(all_sites_data):,}")
print(f"   - Final features: {len(feature_names):,}")
print(f"   - Date range: {all_sites_data['datetime'].min().strftime('%Y-%m-%d')} to {all_sites_data['datetime'].max().strftime('%Y-%m-%d')}")

print(f"\n2. MODEL PERFORMANCE:")
print(f"   Best O3 Model: {best_o3_model_name}")
print(f"   - R²: {o3_comparison.iloc[0]['r2']:.4f}")
print(f"   - RMSE: {o3_comparison.iloc[0]['rmse']:.4f}")
print(f"   - MAE: {o3_comparison.iloc[0]['mae']:.4f}")

print(f"   Best NO2 Model: {best_no2_model_name}")
print(f"   - R²: {no2_comparison.iloc[0]['r2']:.4f}")
print(f"   - RMSE: {no2_comparison.iloc[0]['rmse']:.4f}")
print(f"   - MAE: {no2_comparison.iloc[0]['mae']:.4f}")

print(f"\n3. FORECASTING CAPABILITY:")
print(f"   - Forecast horizon: 24-48 hours")
print(f"   - Prediction interval: Hourly")
print(f"   - Confidence intervals: 90%")
print(f"   - Target pollutants: O3, NO2")

print(f"\n4. OUTPUT FILES:")
print(f"   - Model comparisons: {results_dir}")
print(f"   - Trained models: {models_dir}")
print(f"   - Forecasts: {results_dir / 'forecasts'}")
print(f"   - Evaluation reports: {results_dir}")

print("\n" + "=" * 60)
print("WORKFLOW COMPLETED SUCCESSFULLY!")
print("=" * 60)

logger.info("Air quality forecasting workflow completed successfully")